In [2]:
import numpy as num
import pylab as lab
import pandas as pan
%matplotlib inline

In [3]:
data=pan.read_csv("../golf.csv")
data=num.array(data)
data,data.shape

(array([['sunny', 'hot', 'high', False, 'no'],
        ['sunny', 'hot', 'high', True, 'no'],
        ['overcast', 'hot', 'high', False, 'yes'],
        ['rainy', 'mild', 'high', False, 'yes'],
        ['rainy', 'cool', 'normal', False, 'yes'],
        ['rainy', 'cool', 'normal', True, 'no'],
        ['overcast', 'cool', 'normal', True, 'yes'],
        ['sunny', 'mild', 'high', False, 'no'],
        ['sunny', 'cool', 'normal', False, 'yes'],
        ['rainy', 'mild', 'normal', False, 'yes'],
        ['sunny', 'mild', 'normal', True, 'yes'],
        ['overcast', 'mild', 'high', True, 'yes'],
        ['overcast', 'hot', 'normal', False, 'yes'],
        ['rainy', 'mild', 'high', True, 'no']], dtype=object), (14, 5))

In [4]:
def feature_names(data):
    m,n=num.shape(data)
    names=[]
    for count in range(n):
        example=[]
        for feature in data[:,count]:
            if not(feature in example):
                example.append(feature)
        names.append(example)
    return(names)

In [5]:
namelist=feature_names(data)
namelist

[['sunny', 'overcast', 'rainy'],
 ['hot', 'mild', 'cool'],
 ['high', 'normal'],
 [False, True],
 ['no', 'yes']]

In [6]:
def count_name(data,col,name):
    count=0
    for names in data[:,col]:
        if name==names:
            count=count+1
    return count

In [7]:
count_name(data,4,'yes')

9

In [8]:
def entropy(data,feature_list):
    m,n=data.shape
    entropy_val=0
    for name in feature_list[n-1]:
        countN=count_name(data,n-1,name)
        if countN==0:
            continue;
        prop=float(countN)/m
        entropy_val=entropy_val+(-prop*num.log2(prop))
    return entropy_val
    

In [9]:
en=entropy(data,namelist)
print(en)

0.9402859586706311


In [10]:
def split_data(data,col,feature_list):
    name_list=feature_list[col]
    output_split=[]
    for name in name_list:
        split_temp=[]
        for data_item in data:
            if(name in data_item):
                split_temp.append(data_item)
        split_temp=num.array(split_temp)
        output_split.append(split_temp)
    return num.array(output_split)

In [11]:
splited=split_data(data,1,namelist)
splited

array([array([['sunny', 'hot', 'high', False, 'no'],
       ['sunny', 'hot', 'high', True, 'no'],
       ['overcast', 'hot', 'high', False, 'yes'],
       ['overcast', 'hot', 'normal', False, 'yes']], dtype=object),
       array([['rainy', 'mild', 'high', False, 'yes'],
       ['sunny', 'mild', 'high', False, 'no'],
       ['rainy', 'mild', 'normal', False, 'yes'],
       ['sunny', 'mild', 'normal', True, 'yes'],
       ['overcast', 'mild', 'high', True, 'yes'],
       ['rainy', 'mild', 'high', True, 'no']], dtype=object),
       array([['rainy', 'cool', 'normal', False, 'yes'],
       ['rainy', 'cool', 'normal', True, 'no'],
       ['overcast', 'cool', 'normal', True, 'yes'],
       ['sunny', 'cool', 'normal', False, 'yes']], dtype=object)],
      dtype=object)

In [12]:
def IG(data,col):
    M,n=data.shape
    feature_list=feature_names(data)
    en=entropy(data,feature_list)
    splitted_data=split_data(data,col,feature_list)
    E=0
    for item in splitted_data:
        item=num.array(item)
        m,n=num.shape(item)
        prop=float(m)/M
        E=E+prop*entropy(item,feature_list)
    return en-E
        

In [13]:
IG(data,0)

0.24674981977443933

In [14]:
entropy(splited[1],namelist)

0.9182958340544896

In [15]:
count_name(splited[1],4,'no')

2

In [16]:
def decisionTree(data):
    m,n=data.shape
    list_name=feature_names(data)
    out_list=[]
    entropyD=entropy(data,list_name)
    if(entropyD==0):
        name=data[m-1,n-1]
        count=count_name(data,n-1,name)
        out_list.append("Halt")
        out_list.append(name)
        #out_list.append(count)
        #return(out_list)
    else:
        feat=-1
        temp=-1
        for number in range(n-1):
            in_gain=IG(data,number)
            if(in_gain !=0 and in_gain > temp):
                feat=number
                temp=in_gain
        #out_list.append(feat)
        data_list=split_data(data,feat,list_name)
        for data1 in data_list:
            list2=[]
            list2.append(data1[0][feat])
            list2.append(decisionTree(data1))
            out_list.append(list2)
    return out_list
            
    
    


In [19]:
decisionList=decisionTree(data)

In [20]:
decisionList

[['sunny', [['high', ['Halt', 'no']], ['normal', ['Halt', 'yes']]]],
 ['overcast', ['Halt', 'yes']],
 ['rainy', [[False, ['Halt', 'yes']], [True, ['Halt', 'no']]]]]